### Multi Query Engine ###

In [1]:
from config import set_environment
set_environment()

In [2]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
from llama_index.core import Settings
from llama_index.core.response.notebook_utils import display_response, display_source_node

from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import get_response_synthesizer

INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


In [5]:
# Various Test Docs

paul_graham = "data/test/paul_graham_essay.txt"

aetna_policy = "data/benefits_qa_store/AetnaSBC.pdf"
company_policy = "data/benefits_qa_store/Company - SPD.pdf"


In [6]:
# Node Parser
chunk_size = 1024
chunk_overlap = 20

# Retriever Settings
similarity_top_k = 2

# Context Post Processor Settings
required_key_words = [""]
excluded_key_words = [""]
similarity_cutoff = 0.2

# Response Settings
response_mode = "tree_summarize" # "refine", "tree_summarize"

# Source Node Display Length
source_length = 200

# Document 
document_list = [company_policy]

# Query

query = "what coverage do you offer for life, dental, and vision"

In [7]:
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large",dimensions=512,)
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

#Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [8]:
Settings.llm = OpenAI(temperature=0, model="gpt-4")

In [9]:
documents = SimpleDirectoryReader(input_files=document_list).load_data()

In [10]:
node_parser = SentenceSplitter(chunk_size=chunk_size, chunk_overlap = chunk_overlap)
nodes = node_parser.get_nodes_from_documents(documents)
# set node ids to be a constant
for idx, node in enumerate(nodes):
    node.id_ = f"node-{idx}"

In [11]:
index = VectorStoreIndex(nodes, embed_model=Settings.embed_model, show_progress=True)

Generating embeddings:   0%|          | 0/171 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [12]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=similarity_top_k
)

In [13]:
node_postprocessors = [
    #KeywordNodePostprocessor(
    #   required_keywords=required_key_words, exclude_keywords=excluded_key_words
    #),
    SimilarityPostprocessor(similarity_cutoff=similarity_cutoff) 
]

In [14]:
response_synthesizer = get_response_synthesizer(response_mode = response_mode)

In [15]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=node_postprocessors
)

In [16]:
query = "what coverage do you offer for life, dental, and vision"

In [17]:
response = query_engine.query(query)
display_response(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


**`Final Response:`** The coverage for life, dental, and vision is as follows:

For dental coverage, the plan pays benefits up to either the total amount of your allowable expenses, based on MetLife’s Reasonable and Customary (R&C) rates, or the Plan’s limits, whichever is less. MetLife uses certain rules to determine which plan is primary and which is secondary, such as coordination of benefits, dependent status, and the length of time a person has been covered.

For vision coverage, the plan offers two options: Vision Plan I and Vision Plan II. Vision Plan I allows you to obtain glasses (frame & lenses) or contacts once every calendar year. Vision Plan II allows you to obtain glasses (frame & lenses) or contacts twice every calendar year and provides a higher level of coverage for frames or elective contact lenses. You can access your vision benefits by finding a VSP doctor in your area. 

The document does not provide specific information about life coverage.

In [18]:
retrievals = retriever.retrieve(query)
for n in retrievals:
    display_source_node(n, source_length=source_length)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


**Node ID:** node-116<br>**Similarity:** 0.5627185666598347<br>**Text:** 2024 Oracle America, Inc. Flexible Benefit Plan Document and SPD                                                                                                                                     ...<br>

**Node ID:** node-117<br>**Similarity:** 0.5376066007827193<br>**Text:** 2024 Oracle America, Inc. Flexible Benefit Plan Document and SPD                                                                                                                                     ...<br>

In [18]:
# LLM (gpt-3.5)
gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")

# LLM (gpt-4)
gpt4 = OpenAI(temperature=0, model="gpt-4")

In [19]:
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)

In [20]:
# gpt-4
step_decompose_transform = StepDecomposeQueryTransform(llm=gpt4, verbose=True)

In [21]:
index_summary = "Used to answer questions about benefits coverage"

In [22]:
# set Logging to DEBUG for more detailed outputs
from llama_index.core.query_engine import MultiStepQueryEngine


multi_query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform,
    index_summary=index_summary,
)
response_gpt4 = multi_query_engine.query(query)
   

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Current query: what coverage do you offer for life, dental, and vision
> New query: What life coverage do you offer?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Current query: what coverage do you offer for life, dental, and vision
> New query: What dental and vision coverage do you offer?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 

In [23]:
display_response(response_gpt4)

**`Final Response:`** The life coverage offered includes Group Term Life Insurance, which provides benefits to your named beneficiary in the event of your death. There are pre-tax and after-tax life insurance coverage options, with a combined maximum life insurance amount of $2,550,000.

Dental coverage is part of the Flexible Benefit Plan, with benefits determined based on MetLife’s Reasonable and Customary (R&C) rates or the Plan’s limits, whichever is less. 

For vision coverage, there are two plans: Vision Plan I and Vision Plan II. Vision Plan I allows you to obtain glasses or contacts once every calendar year, while Vision Plan II allows you to obtain glasses or contacts twice every calendar year and provides a higher level of coverage for frames or elective contact lenses.

In [24]:
import pprint

In [25]:
sub_qa = response_gpt4.metadata["sub_qa"]

for t in sub_qa:
    pprint.pprint(t[0])


'What life coverage do you offer?'
'What dental and vision coverage do you offer?'


In [26]:
for t in sub_qa:
    pprint.pprint(t[1].response)

('The life coverage offered includes Group Term Life Insurance, which provides '
 'benefits to your named beneficiary in the event of your death. You are '
 'required to elect group term life insurance coverage. There are pre-tax and '
 'after-tax life insurance coverage options. The pre-tax options allow you to '
 'pay for coverage using pre-tax dollars, with a maximum coverage amount of '
 '$50,000. The after-tax options allow you to pay for coverage using after-tax '
 'dollars, with coverage options being multiples of your Annual Benefits '
 'Compensation (1 – 6 times) up to the maximum coverage amount of $2,500,000. '
 'The combined pre-tax and after-tax maximum life insurance amount is '
 '$2,550,000. If you are a newly eligible employee making your first election, '
 'you may choose any life insurance amount (up to the maximum) without medical '
 'review. Death benefits received are generally not taxable.')
('Oracle America, Inc. offers both dental and vision coverage as part of 

In [27]:
print(type(sub_qa))

<class 'list'>
